In [ ]:
"""
Hit-based acquisition.

Acoustic Emission measurements are usually hit-based. Only when a defined threshold is crossed,
data is acquired. The acquired data is mainly influenced by two parameters:
- threshold amplitude
- duration discrimination time (DDT); if no threshold crossings are detected for the length of DDT,
  the end of the hit is determined

In [ ]:
"""""
Uniaxial tension and compression of porous Mg-Ho alloys were measured using an Instron 5969 Universal Testing Machine. 
The samples for tension experiments were dog-bone-shaped with a gauge range of 10 mm × 3 mm × 1 mm. 
The compression samples were cylinders with high H = 15 mm and diameter φ = 6 mm. 
Tension and compression experiments were performed at room temperature with velocity 10−3 mm/min.

A piezoelectric sensor (Vallen-Systeme GmbH) with a frequency band of 200–800 kHz was attached to the sample surface. 
AE signals were first pre-amplified by 43 dB and then transferred to the AMSY-6 AE-measurement system (Vallen-Systeme GmbH) using a frequency range 95–850 kHz. 
The amplitude A is recorded in dB which follows the expression dB = [20 Log (|Vsensor|/1 μV)], 
where Vsensor is the peak voltage output by sensor, and the brackets round the value to its nearest integer in dB.

An AE signal is defined as a ‘burst’ in the noise spectrum. 
The start of a burst is determined by a first threshold crossing, 
and ended at second threshold crossing or when Duration Discrimination time (DDT, defines a time period in which no threshold crossing must occur in order that an end of hit is determined) expired without any threshold crossing. 
A threshold of 21.1 dB was determined by prior rubber experiments to evaluate the internal noise of the experimental arrangement. 
We changed the DDT from 50 μs to 1000 μs, and found that DDT ~ 300 μs fits the porous collapse well, while DDT ~ 50 μs fits the dislocation movements well. 
In order to capture both porous collapse and dislocation movements, the value of DDT = 100 μs was chosen. 
The AE energy is the integral of the square of the output voltage over the burst interval. 
The energy is then electronically calibrated into atto Joules, aJ.

In order to ensure that all the AE signals collected during the tension/compression experiment stem from the sample, 
we performed noise separation measurement. We first investigated the noise of the full experiment using a dummy rubber sample with the same detector configuration as in the alloy experiment (see Supplementary Fig. S2).
We also measured the noise of the instrument by attaching the detector to the sample grips. 
The distribution of background noise follows approximately a log normal distribution (see Supplementary Fig. S3) while the maximum noise energy is less than 0.2 aJ.

In [ ]:
"""
Additionally, status data can be acquired in defined intervals.

The following example shows a simple setup to acquire hits and status data with a linWave
device. Hit data (AERecord) and transient data (TRRecord) are returned from different functions but
can be merged by matching the transient recorder index (trai) field in both records.
"""

In [ ]:
import argparse  # argparse is the “recommended command-line parsing module in the Python standard library.” 
#It’s what you use to get command line arguments into your program.

import asyncio
import logging
from dataclasses import asdict, dataclass
from typing import Dict

import numpy as np

from waveline import LinWave, AERecord, TRRecord

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
@dataclass
class HitRecord(AERecord):
    """All fields from AERecord + fields for transient data."""

    samples: int
    data: np.ndarray

In [ ]:
async def merge_ae_tr_records(async_generator):
    """Helper function to merge matching AERecords and TRRecords (same trai)."""
    dict_ae: Dict[int, AERecord] = {}
    dict_tr: Dict[int, TRRecord] = {}

    async for record in async_generator:
        if isinstance(record, AERecord):
            if record.trai == 0:  # status data or hit without transient data -> return directly
                yield record
            else:
                dict_ae[record.trai] = record  # store in buffer to merge later
        if isinstance(record, TRRecord):
            dict_tr[record.trai] = record  # store in buffer to merge later

        # try to match and return merged HitRecords
        trais_ae = set(dict_ae.keys())
        trais_tr = set(dict_tr.keys())
        trais_match = trais_ae.intersection(trais_tr)
        for trai in trais_match:
            ae_record = dict_ae.pop(trai)
            tr_record = dict_tr.pop(trai)
            yield HitRecord(
                **asdict(ae_record),
                samples=tr_record.samples,
                data=tr_record.data,
            )

In [ ]:
async def main(ip: str):
    async with LinWave(ip) as lw:
        print(await lw.get_info())

        await lw.set_channel(channel=0, enabled=True)  # enabled all channels
        await lw.set_range(channel=0, range_volts=0.05)  # set input range to 50 mV
        await lw.set_filter(channel=0, highpass=100e3, lowpass=450e3)  # 100-450 kHz bandpass
        await lw.set_continuous_mode(channel=0, enabled=False)  # -> hit-based
        await lw.set_ddt(channel=0, microseconds=400)  # set duration discrimination time to 400 µs
        await lw.set_status_interval(channel=0, seconds=2)  # generate status data every 2 seconds
        await lw.set_threshold(channel=0, microvolts=1_000)  # 1000 µV = 60 dB(AE)
        await lw.set_tr_enabled(channel=0, enabled=True)  # enable transient data recording
        await lw.set_tr_decimation(channel=0, factor=5)  # set decimation factor for transient data, 10 MHz / 10 = 1 MHz
        await lw.set_tr_pretrigger(channel=0, samples=200)  # 200 pre-trigger samples
        await lw.set_tr_postduration(channel=0, samples=200)  # 0 post-duration samples

        print(await lw.get_setup(channel=1))
        print(await lw.get_setup(channel=2))

        async for record in merge_ae_tr_records(lw.acquire()):
            print(record)

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="linwave_ae")
    parser.add_argument("ip", help="IP address of linWave device")
    args = parser.parse_args()

    try:
        asyncio.run(main(args.ip))
    except KeyboardInterrupt:
        ...

In [ ]:
"""   Example 
import argparse
parser = argparse.ArgumentParser(description='Videos to images')
parser.add_argument('indir', type=str, help='Input dir for videos')
parser.add_argument('outdir', type=str, help='Output dir for image')
args = parser.parse_args()
print(args.indir)